In [18]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
# READ DATA
df = pd.read_csv('SerieA.csv', sep=',', header=0)
data = df.to_numpy()  # nd-array of size (381,6)

In [20]:
# SET UP TEAMS
nb_matches = len(data)
teams_names = np.unique(data[:, 2:4])  # array of names of teams
nb_teams = len(teams_names)
mu = np.zeros(nb_teams)
var = np.ones(nb_teams)
teams = np.stack((teams_names, mu, var))

In [21]:
# DO GIBBS
def importance(s1, s2, y=1, s_e=1):
    pr = 1-stats.norm(s1-s2, s_e**0.5).cdf(0) if y > 0 else stats.norm(s1-s2, s_e**0.5).cdf(0)
    return pr

def approx_gaussian(data, importances):
    mean = np.average(data, weights=importances)
    variance = np.average((data-mean)**2, weights=importances)
    return mean, variance

## Question 6

In [20]:
num_samples = 3000
drops = 1000

# initialize
s1 = 0
s1_w = 1

for game_ind in range(nb_matches):
    print(game_ind)
    # result
    if data[game_ind, 4] - data[game_ind, 5] > 0:
        y = 1
    elif data[game_ind, 4] - data[game_ind, 5] < 0:
        y = -1
    else:
        continue  # exit loop since we don't wanna update the skills because it's draw

    # set up so that we can use the code we already have
    team_1 = data[game_ind, 2]
    ind_team_1 = np.where(teams[0, :] == team_1)
    m1 = float(teams[1, ind_team_1])
    v1 = float(teams[2, ind_team_1])

    team_2 = data[game_ind, 3]
    ind_team_2 = np.where(teams[0, :] == team_2)
    m2 = float(teams[1, ind_team_2])
    v2 = float(teams[2, ind_team_2])

    posterior = []
    post_weights = []

    for _ in range(num_samples):
        s2 = stats.norm.rvs(m2, v2 ** 0.5)
        s2_w = importance(s1, s2, y)

        posterior.append((s1, s2))
        post_weights.append((s1_w, s2_w))

        s1 = stats.norm.rvs(m1, v1 ** 0.5)
        s1_w = importance(s1, s2, y)

    posterior = np.array(posterior)
    post_weights = np.array(post_weights)

    m1, v1 = approx_gaussian(posterior[drops:, 0], post_weights[drops:, 0])
    m2, v2 = approx_gaussian(posterior[drops:, 1], post_weights[drops:, 1])

    # UPDATE THE ARRAY WITH SKILLS
    teams[1, ind_team_1] = m1
    teams[2, ind_team_1] = v1
    teams[1, ind_team_2] = m2
    teams[2, ind_team_2] = v2


print(teams)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Question 7

In [5]:
def predict_match_result(m1, m2, v1, v2, m=5000, s_e=1):
    # generate m skill samples from posterior distribution
    s1_samples = stats.norm.rvs(m1, v1 ** 0.5, size=m)
    s2_samples = stats.norm.rvs(m2, v2 ** 0.5, size=m)

    # and approximate prediction with Monte Carlo approach
    p_loss = np.sum(stats.norm.cdf(-(s1_samples - s2_samples) / s_e)) / m

    if p_loss <= 0.5: # if win
        return 1
    else:             # if loss
        return -1
    

### with $\sigma_t$ (s_e) fixed 

In [22]:
num_samples = 3000
drops = 1000

# initialize
s1 = 0
s1_w = 1

correct_guesses = 0
total_guesses = 0

for game_ind in range(nb_matches):
    # result
    if data[game_ind, 4] - data[game_ind, 5] > 0:
        y = 1
    elif data[game_ind, 4] - data[game_ind, 5] < 0:
        y = -1
    else:
        continue  # exit loop since we don't wanna update the skills because it's draw

    # set up so that we can use the code we already have
    team_1 = data[game_ind, 2]
    ind_team_1 = np.where(teams[0, :] == team_1)
    m1 = float(teams[1, ind_team_1])
    v1 = float(teams[2, ind_team_1])

    team_2 = data[game_ind, 3]
    ind_team_2 = np.where(teams[0, :] == team_2)
    m2 = float(teams[1, ind_team_2])
    v2 = float(teams[2, ind_team_2])

    # make prediction (before updating posterior)
    y_predict = predict_match_result(m1, m2, v1, v2)
    if y_predict == y:
        correct_guesses += 1
    total_guesses += 1

    print(f"Game #{game_ind}: y predict = {y_predict}, y true = {y}")

    # update posterior
    posterior = []
    post_weights = []

    for _ in range(num_samples):
        s2 = stats.norm.rvs(m2, v2 ** 0.5)
        s2_w = importance(s1, s2, y)

        posterior.append((s1, s2))
        post_weights.append((s1_w, s2_w))

        s1 = stats.norm.rvs(m1, v1 ** 0.5)
        s1_w = importance(s1, s2, y)

    posterior = np.array(posterior)
    post_weights = np.array(post_weights)

    m1, v1 = approx_gaussian(posterior[drops:, 0], post_weights[drops:, 0])
    m2, v2 = approx_gaussian(posterior[drops:, 1], post_weights[drops:, 1])

    # UPDATE THE ARRAY WITH SKILLS
    teams[1, ind_team_1] = m1
    teams[2, ind_team_1] = v1
    teams[1, ind_team_2] = m2
    teams[2, ind_team_2] = v2


print("r = ", correct_guesses / total_guesses, ", which is better than 0.5", sep="")

    

Game #0: y predict = -1, y true = -1
Game #1: y predict = -1, y true = -1
Game #2: y predict = -1, y true = -1
Game #3: y predict = -1, y true = 1
Game #5: y predict = 1, y true = 1
Game #6: y predict = -1, y true = -1
Game #7: y predict = 1, y true = 1
Game #8: y predict = 1, y true = 1
Game #9: y predict = 1, y true = 1
Game #10: y predict = 1, y true = 1
Game #11: y predict = -1, y true = 1
Game #13: y predict = -1, y true = 1
Game #15: y predict = 1, y true = 1
Game #18: y predict = 1, y true = 1
Game #19: y predict = -1, y true = -1
Game #20: y predict = -1, y true = -1
Game #21: y predict = -1, y true = 1
Game #22: y predict = 1, y true = 1
Game #23: y predict = 1, y true = 1
Game #24: y predict = -1, y true = 1
Game #25: y predict = 1, y true = 1
Game #27: y predict = 1, y true = -1
Game #28: y predict = 1, y true = -1
Game #29: y predict = 1, y true = 1
Game #30: y predict = -1, y true = -1
Game #33: y predict = 1, y true = 1
Game #34: y predict = -1, y true = 1
Game #35: y pre

### with $\sigma_t$ obtained from Marginal Likelihood optimization

In [41]:
# # s_e = argmax(Marginal Likelihood for data given by array t)
# def optimal_variance(t, m1, m2, v1, v2):
#     s_e = np.sum( (t - m1 + m2)**2 - (v1 + v2) ) / len(t)
#     # bad way to treat this case, should consider the cause
#     if s_e <= 0:
#         return 1
#     else:
#         return s_e

from scipy import optimize as opt
# s_e = argmax(Marginal Likelihood for data given by array t)
def optimal_variance(t, m1, m2, v1, v2):
    # marginal likelihood for optimization
    def cost_func(sigma):
        s = sigma + v1 + v2 # sigma = \sigma_t^2
        return np.sum( np.log(s) + (t - (m1 - m2))**2 / s )
    
    # derivative of marginal likelihood for optimization
    def cost_func_grad(sigma):
        s = sigma + v1 + v2
        return np.sum( s**(-1) - (t - (m1 - m2))**2 * s**(-2) )

    opt_res = opt.minimize(cost_func, x0=1, method='cg', jac=cost_func_grad)
    s_e = float(opt_res.x)
    print("\sigma_t =", s_e)
    return s_e

In [42]:
num_samples = 3000
drops = 1000

# initialize
s1 = 0
s1_w = 1

correct_guesses = 0
total_guesses = 0

t_arr = np.empty((0))
m_arr = np.empty((0, 2))
v_arr = np.empty((0, 2))

for game_ind in range(nb_matches):
    # result
    if data[game_ind, 4] - data[game_ind, 5] > 0:
        y = 1
    elif data[game_ind, 4] - data[game_ind, 5] < 0:
        y = -1
    else:
        continue  # exit loop since we don't wanna update the skills because it's draw

    # set up so that we can use the code we already have
    team_1 = data[game_ind, 2]
    ind_team_1 = np.where(teams[0, :] == team_1)
    m1 = float(teams[1, ind_team_1])
    v1 = float(teams[2, ind_team_1])

    team_2 = data[game_ind, 3]
    ind_team_2 = np.where(teams[0, :] == team_2)
    m2 = float(teams[1, ind_team_2])
    v2 = float(teams[2, ind_team_2])

    t = float(data[game_ind, 4]) - float(data[game_ind, 5]) # obeserved data: s1 - s2

    # for computing optimal variance s_e
    t_arr = np.append(t_arr, t)
    m_arr = np.append(m_arr, [[m1, m2]], axis=0)
    v_arr = np.append(v_arr, [[v1, v2]], axis=0)

    # make prediction (before updating posterior)
    s_e = optimal_variance(t_arr, m_arr[:, 0], m_arr[:, 1], v_arr[:, 0], v_arr[:, 1]) # compute optimal variance given observed results
    y_predict = predict_match_result(m1, m2, v1, v2, s_e=s_e)
    if y_predict == y:
        correct_guesses += 1
    total_guesses += 1

    print(f"Game #{game_ind}: y predict = {y_predict}, y true = {y}")

    # update posterior
    posterior = []
    post_weights = []

    for _ in range(num_samples):
        s2 = stats.norm.rvs(m2, v2 ** 0.5)
        s2_w = importance(s1, s2, y, s_e=s_e)

        posterior.append((s1, s2))
        post_weights.append((s1_w, s2_w))

        s1 = stats.norm.rvs(m1, v1 ** 0.5)
        s1_w = importance(s1, s2, y, s_e=s_e)

    posterior = np.array(posterior)
    post_weights = np.array(post_weights)

    m1, v1 = approx_gaussian(posterior[drops:, 0], post_weights[drops:, 0])
    m2, v2 = approx_gaussian(posterior[drops:, 1], post_weights[drops:, 1])

    # UPDATE THE ARRAY WITH SKILLS
    teams[1, ind_team_1] = m1
    teams[2, ind_team_1] = v1
    teams[1, ind_team_2] = m2
    teams[2, ind_team_2] = v2


print("r = ", correct_guesses / total_guesses, ", which is better than 0.5", sep="")

    

$\sigma_t =$ 7.541820323453388
Game #0: y predict = -1, y true = -1
$\sigma_t =$ 3.5796934406488616
Game #1: y predict = -1, y true = -1
$\sigma_t =$ 2.4250971755451527
Game #2: y predict = -1, y true = -1


KeyboardInterrupt: 